## LOOCV on DementiaBank 

This code was run using:
- gluonnlp                  0.8.1                    pypi_0    pypi
- mxnet-cu100mkl            1.5.0                    pypi_0    pypi
- nltk                      3.4.5                    py37_0  
- sacremoses                0.0.33                   pypi_0    pypi

In [2]:
#get pretrained embeddings (sgns with subwords, trained on wikipedia)
!cd totp_data; wget https://www.dropbox.com/s/2bev8m6uahf15r1/wikipedia_sgns_embeddings.bin

--2020-07-12 16:58:01--  https://www.dropbox.com/s/2bev8m6uahf15r1/wikipedia_sgns_embeddings.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2bev8m6uahf15r1/wikipedia_sgns_embeddings.bin [following]
--2020-07-12 16:58:01--  https://www.dropbox.com/s/raw/2bev8m6uahf15r1/wikipedia_sgns_embeddings.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9955a9f083e5bc7dd0f77e9b29.dl.dropboxusercontent.com/cd/0/inline/A7bP0SM3bdUyLn-IhkjI7vlMPy0qfi2Q4zt7BlWDY9d5Scf5FDYbZUlY5IgJLhE0WOi87xbGoV2_IXp8f7ADKXwwTCYXRmipuNu3KsYOaBX-_t4-pCNBlsHPo2qWdH4K38o/file# [following]
--2020-07-12 16:58:01--  https://uc9955a9f083e5bc7dd0f77e9b29.dl.dropboxusercontent.com/cd/0/inline/A7bP0SM3bdUyLn-IhkjI7vlMPy0qfi2Q4zt7BlWDY9d5Scf5FDYbZUlY5IgJLhE

In [125]:
#before uncommenting running the cells below, you will need to download the DementiaBank corpus, and place the 
#Control and Dementia .cha transcripts of the cookie thefts into the Control and Dementia folders of totp_data
#!mkdir totp_data/txtControl; mkdir totp_data/txtDementia
#!cat totp_data/*.py
#!python totp_data/chat2txt.py --chatdir 'totp_data/Control/*.cha' --output totp_data/txtControl
#!python totp_data/chat2txt.py --chatdir 'totp_data/Dementia/*.cha' --output totp_data/txtDementia

In [111]:
!head totp_data/txtControl/002-0.txt

 the scene is in the in the kitchen 
 the mother is wiping dishes and the water is running on the floor 
 a child is trying to get a boy is trying to get cookies out of a jar and he's about to tip over on a stool 
 the little girl is reacting to his falling 
 it seems to be summer out 
 the window is open 
 the curtains are blowing 
 it must be a gentle breeze 
 there's grass outside in the garden 
 mother's finished certain of the the dishes 


In [112]:
!ls totp_data/txtControl | wc -l
!ls totp_data/txtDementia | wc -l

242
309


In [113]:
#http://gluon-nlp.mxnet.io/master/examples/language_model/language_model.html
import warnings
warnings.filterwarnings('ignore')
import glob
import time
import math
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.utils import download
import gluonnlp as nlp
import pandas as pd
#Collect data
_dir='./totp_data/'

control_text = [open(filename).read() for filename in glob.glob(_dir+"txtControl/*.txt")]
control_names = [filename.replace('./totp_data/txtControl/','') for filename in glob.glob(_dir+"txtControl/*.txt")]
dementia_text =[open(filename).read() for filename in glob.glob(_dir+"txtDementia/*.txt")]
dementia_names=[filename.replace('./totp_data/txtDementia/','') for filename in glob.glob(_dir+"txtDementia/*.txt")]

In [114]:
import nltk
import numpy as np
control_frame=pd.DataFrame(list(zip(control_names,control_text)),columns=['file','text'])
dementia_frame=pd.DataFrame(list(zip(dementia_names,dementia_text)),columns=['file','text'])
control_frame['label']=np.zeros(len(control_frame))
dementia_frame['label']=np.ones(len(dementia_frame))

In [115]:
control_frame.head()

,file,text,label
0,021-1.txt,okay \n many or the mother's washing the dish...,0.0
1,208-1.txt,i see the lady drying the dishes \n and the w...,0.0
2,071-0.txt,you mean this house \n oh whatever \n uhhuh \...,0.0
3,678-0.txt,well the sink's overflowing \n and the stool'...,0.0
4,243-1.txt,i see a boy getting in the in the cookie jar ...,0.0


In [116]:
conids=set()
demids=set()
for id in control_frame['file']:
    conids.add(id[:3])
for id in dementia_frame['file']:
    demids.add(id[:3])

In [117]:
print('Participants: ',len(conids),' controls and ',len(demids),' cases (dementia)')
print('Transcripts : ',len(control_frame['file']),' controls and ',len(dementia_frame['file']),' cases (dementia)')

Participants:  99  controls and  194  cases (dementia)
Transcripts :  242  controls and  309  cases (dementia)


In [118]:
dementia_text[0]

" okay \n the mother is washing dishes \n and the water is flowing from the sink \n she's looking out the window \n the son or the boy's on a stool which he's about to fall off stealing cookies from the cookie jar and giving it to probably his little sister \n there are two cups and a dish on the sink counter \n and the lady at the sink has a dish and a wash and a dry cloth \n do you want me to describe her clothes \n oh \n"

## LOOCV

In [122]:
#methods to train and evaluate the RNN - very minor modifications to standard GluonNLP routines
def detach(hidden):
    if isinstance(hidden, (tuple, list)):
        hidden = [detach(i) for i in hidden]
    else:
        hidden = hidden.detach()
    return hidden

# Note that ctx is short for context
def mevaluate(model, data_source, batch_size, ctx):
    total_L = 0.0
    ntotal = 0
    hidden = model.begin_state(
        batch_size=batch_size, func=mx.nd.zeros, ctx=ctx)
    for i, (data, target) in enumerate(data_source):
        data = data.as_in_context(ctx)
        target = target.as_in_context(ctx)
        output, hidden = model(data, hidden)
        hidden = detach(hidden)
        L = loss(output.reshape(-3, -1), target.reshape(-1))
        total_L += mx.nd.sum(L).asscalar()
        #ntotal += L.size - only count non-zero results so zero paddiing doesn't increase the denominator
        ntotal += (L != 0).sum().asnumpy()[0]
    if (ntotal > 0):
        return total_L / ntotal
    else:
        return 0


# Function for actually training the model
def mtrain(model, train_data, val_data, test_data, epochs, lr):
    best_val = float("Inf")
    start_train_time = time.time()
    parameters = model.collect_params().values()

    print('len train data ',len(train_data))
    
    for epoch in range(epochs):
        total_L = 0.0
        start_epoch_time = time.time()
        start_log_interval_time = time.time()
        hiddens = [model.begin_state(batch_size//len(context), func=mx.nd.zeros, ctx=ctx)
                   for ctx in context]

        for i, (data, target) in enumerate(train_data):
            data_list = gluon.utils.split_and_load(data, context,
                                                   batch_axis=1, even_split=True)
            target_list = gluon.utils.split_and_load(target, context,
                                                     batch_axis=1, even_split=True)
            hiddens = detach(hiddens)
            L = 0
            Ls = []

            with autograd.record():
                for j, (X, y, h) in enumerate(zip(data_list, target_list, hiddens)):
                    output, h = model(X, h)
                    batch_L = loss(output.reshape(-3, -1), y.reshape(-1,))
                    L = L + batch_L.as_in_context(context[0]) / (len(context) * X.size)
                    Ls.append(batch_L / (len(context) * X.size))
                    hiddens[j] = h
            L.backward()
            grads = [p.grad(x.context) for p in parameters for x in data_list]
            gluon.utils.clip_global_norm(grads, grad_clip)

            trainer.step(1)

            total_L += sum([mx.nd.sum(l).asscalar() for l in Ls])

            if i % log_interval == 0 and i > 0:
                cur_L = total_L / log_interval
                print('[Epoch %d Batch %d/%d] loss %.2f, ppl %.2f, '
                      'throughput %.2f samples/s'%(
                    epoch, i, len(train_data), cur_L, math.exp(cur_L),
                    batch_size * log_interval / (time.time() - start_log_interval_time)))
                total_L = 0.0
                start_log_interval_time = time.time()

        mx.nd.waitall()

        print('[Epoch %d] throughput %.2f samples/s'%(
                    epoch, len(train_data)*batch_size / (time.time() - start_epoch_time)))

        val_L = mevaluate(model, val_data, batch_size, context[0])
        print('[Epoch %d] time cost %.2fs, valid loss %.2f, valid ppl %.2f'%(
            epoch, time.time()-start_epoch_time, val_L, math.exp(val_L)))

        if val_L < best_val:
            best_val = val_L
            test_L = mevaluate(model, test_data, batch_size, context[0])
            model.save_parameters('{}_{}-{}.params'.format(model_name, dataset_name, epoch))
            print('test loss %.2f, test ppl %.2f'%(test_L, math.exp(test_L)))
        else:
            lr = lr*0.25
            print('Learning rate now %f'%(lr))
            trainer.set_learning_rate(lr)

    print('Total training throughput %.2f samples/s'%(
                            (batch_size * len(train_data) * epochs) /
                            (time.time() - start_train_time)))

In [121]:
#versions
!conda list | grep "mxnet\|gluon\|nltk\|moses"

# packages in environment at /home/cohenta/anaconda3/envs/gluon37:
gluonnlp                  0.8.1                    pypi_0    pypi
mxnet-cu100mkl            1.5.0                    pypi_0    pypi
nltk                      3.4.5                    py37_0  
sacremoses                0.0.33                   pypi_0    pypi


In [123]:
#tokinzation / batchification 
def tokenize_frame(incoming_frame):
    toks=[]
    for response in incoming_frame['text']:
        toks.extend(moses_tokenizer.tokenize(response.replace('\n',' ')))    
        toks.extend(['<eos>'])
    return toks

def tokenize_frames(train_frame,dementia_frame,test_frame):
    toks=tokenize_frame(train_frame)
    test_toks=tokenize_frame(test_frame)
    dem_toks=tokenize_frame(dementia_frame)
    return toks, dem_toks, test_toks
        
def batchify(trainset,valset,testset,dtrainset,dvalset):
    adasets=[trainset,valset,testset,dtrainset,dvalset]
    ad_train_data, ad_val_data, test_data,d_train_data,d_val_data = [bptt_batchify(dataset) for dataset in adasets]
    return ad_train_data, ad_val_data, d_train_data,d_val_data,test_data

In [126]:
bird_text = [open(filename).read().lower() for filename in glob.glob("totp_data/bird_transcripts/*.txt")]
bird_names = [filename.replace('totp_data/bird_transcripts/','') for filename in glob.glob("totp_data/bird_transcripts/*.txt")]
bird_frame=pd.DataFrame(list(zip(bird_names,bird_text)),columns=['file','text'])

In [127]:
#to use development version of semvecpy - but could also just pip install semvecpy
import sys
#!git clone https://github.com/semanticvectors/semvecpy
sys.path.append('semvecpy')
import semvecpy.vectors.real_vectors as rv

In [131]:
!ls totp_data/wikipedia_sgns_embeddings.bin
!mkdir totp_output

totp_data/wikipedia_sgns_embeddings.bin


In [ ]:
%%capture gluout
#load data
import nltk
import numpy as np
import pandas as pd
import mxnet as mx
import math
from mxnet import gluon, autograd
from mxnet.gluon.utils import download
import gluonnlp as nlp
from gluonnlp.model import train
from sacremoses import MosesTokenizer
moses_tokenizer = MosesTokenizer('en')

dataset_name='DementiaBank'
control_frame=pd.DataFrame(list(zip(control_names,control_text)),columns=['file','text'])
dementia_frame=pd.DataFrame(list(zip(dementia_names,dementia_text)),columns=['file','text'])
control_frame['label']=np.zeros(len(control_frame))
dementia_frame['label']=np.ones(len(dementia_frame))
double_frame=control_frame.append(dementia_frame)

allids=[]

for id in double_frame['file']:
    allids.append(id[:3])

double_frame['id']=allids

#configure models
num_gpus = 1
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else [mx.cpu()]
log_interval = 200
batch_size = 20 * len(context)
test_batch_size = 1* len(context)
lr = 20
epochs = 20
bptt = 10
grad_clip = 0.25

print('Participants: ',len(conids),' controls and ',len(demids),' cases (dementia)')
print('Transcripts : ',len(dementia_frame['file']),' controls and ',len(control_frame['file']),' cases (dementia)')


# Specify the loss function, in this case, cross-entropy with softmax.
loss = gluon.loss.SoftmaxCrossEntropyLoss()

#LOOCV
#earp = 'random' #uncomment for stochastic initiliazation
earp = 'basic'   #uncomment for pretrained embeddings
#earp = 'wiki'   #uncomment for pretrained LM

dataset_name = None
pretrained = False

for earp in ['random','basic','wiki']:

    for e in [1]: #,2,3,4,5,6,7,8,9,10]:   #ten iterations used for paper
        f = open('totp_output/'+str(e)+'gluon_output_'+earp+'.tmp', "w")
        g = open('totp_output/'+str(e)+'bird_output_'+earp+'.tmp',"w")
        sallids=set(allids)
        prefix = 'totp_data/wikipedia_sgns_'
        posfix = 'embeddings.bin'


        for currentid in list(sallids):

            if earp != 'random':
                lr = 5
                if earp == 'wiki':
                    dataset_name = 'wikitext-2'
                    pretrained = True
                else:
                    glovecs = rv.RealVectorStore()
                    glovecs.init_from_file(prefix+earp+posfix)

            test_frame=double_frame[double_frame['file'].str.contains('^'+currentid)]
            train_frame=control_frame[~control_frame['file'].str.contains('^'+currentid)]
            dem_frame=dementia_frame[~dementia_frame['file'].str.contains('^'+currentid)]

            toks, dem_toks, test_toks = tokenize_frames(train_frame,dem_frame,test_frame)
            x=int(.9*len(toks))
            trainset=toks[:x]
            valset=toks[x:]

            x=int(.9*len(dem_toks))
            dtrainset=dem_toks[:x]
            dvalset=dem_toks[x:]

            testset=[]
            testset=test_toks

            vocab = nlp.Vocab(counter=nlp.data.Counter(trainset+dtrainset))
            dvocab = nlp.Vocab(counter=nlp.data.Counter(trainset+dtrainset))

            bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                vocab, bptt, batch_size, last_batch='keep')  #changed from 'discard'

            dbptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                dvocab, bptt, batch_size, last_batch='keep')

            train_data, val_data, test_data = [
                bptt_batchify(x) for x in [trainset, valset, testset]]

            dtrain_data, dval_data, dtest_data = [
                dbptt_batchify(x) for x in [dtrainset, dvalset, testset]]

            model_name = 'standard_lstm_lm_200'
            dmodel_name = 'standard_lstm_lm_200'
            model, vocab = nlp.model.get_model(model_name, vocab=vocab, dataset_name=dataset_name,pretrained=pretrained,ctx=context[0])
            dmodel, dvocab = nlp.model.get_model(dmodel_name, vocab=dvocab, dataset_name=dataset_name,pretrained=pretrained,ctx=context[0])

            # Initialize the model
            if (dataset_name == None):
                dmodel.initialize(mx.init.Xavier(), ctx=context)

            #rebatchify if pretrained (use pretrained vocab)
            if (earp == 'wiki'):
                vocab.padding_token = '<pad>'
                bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                    vocab, bptt, batch_size, last_batch='keep')  #changed from 'discard'
                dbptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                    vocab, bptt, batch_size, last_batch='keep')
                train_data, val_data, test_data = [
                    bptt_batchify(x) for x in [trainset, valset, testset]]
                dtrain_data, dval_data, dtest_data = [
                    dbptt_batchify(x) for x in [dtrainset, dvalset, testset]]
                dvocab = vocab


            # Replace embeddings with pretrained (not necessarily Glove) vectors
            if earp != 'random' and earp != 'wiki':
                    parameters = dmodel.collect_params().values()
                    ps=[]
                    for p in parameters:
                        ps.append(p)

                    print(len(ps[0].list_data()[0].asnumpy()), len(vocab))
                    terms=[]
                    vecs=[]
                    for i in (range(len(vocab))):
                        term = vocab.idx_to_token[i]
                        terms.append(term)
                        if term in glovecs.dict:
                            vecs.append(glovecs.get_vector(term).copy().vector)
                        else:
                            vecs.append(ps[0].list_data()[0][i].asnumpy())
                    ps[0].set_data(np.array(vecs))

            # Initialize the trainer and optimizer and specify some hyperparameters
            trainer = gluon.Trainer(dmodel.collect_params(), 'sgd', {
                'learning_rate': lr,
                'momentum': 0,
                'wd': 0
            })

            #train dementia model
            mtrain(
                dmodel,
                dtrain_data, 
                dval_data,
                dtest_data, 
                epochs=epochs,
                lr=lr)


            if (dataset_name == None):
                model.initialize(mx.init.Xavier(), ctx=context)

            # Replace embeddings with pretrained Glove vectors
            if earp != 'random' and earp != 'wiki':
                    parameters = model.collect_params().values()
                    ps=[]
                    for p in parameters:
                        ps.append(p)

                    print(len(ps[0].list_data()[0].asnumpy()), len(vocab))
                    terms=[]
                    vecs=[]
                    for i in (range(len(vocab))):
                        term = vocab.idx_to_token[i]
                        terms.append(term)
                        if term in glovecs.dict:
                            vecs.append(glovecs.get_vector(term).copy().vector)
                        else:
                            vecs.append(ps[0].list_data()[0][i].asnumpy())
                    ps[0].set_data(np.array(vecs))


            trainer = gluon.Trainer(model.collect_params(), 'sgd', {
                'learning_rate': lr,
                'momentum': 0,
                'wd': 0
            })

            #train control model 
            mtrain(
                model,
                train_data, 
                val_data,
                test_data, 
                epochs=epochs,
                lr=lr)


            mixframe = pd.DataFrame(columns=['xdementia','perplexity','file'])
            cparameters = model.collect_params().values()
            dparameters = dmodel.collect_params().values()
            cps=[]
            for p in cparameters:
                cps.append(p)
            dps=[]
            for p in dparameters:
                dps.append(p)
            #get average of parameters (interrogation by interpolation)
            mix_models = []
            for alpha in [0,.25,.5,.75,1]:
                model_name = 'standard_lstm_lm_200'
                mix_model, vocab = nlp.model.get_model(model_name, vocab=vocab, dataset_name=None)
                mix_model.initialize(mx.init.Xavier(), ctx=context)

                mix_parameters = mix_model.collect_params().values()
                for i,p in enumerate(mix_parameters):
                    cp = cps[i].list_data()[0].asnumpy()
                    dp = dps[i].list_data()[0].asnumpy()
                    nudata = (alpha*dp + (1-alpha)*cp)
                    p.set_data(np.asarray(nudata))

                mix_models.append(mix_model)
           
            #output bird results (interrogation by perturbation)
            for i,row in bird_frame.iterrows():
                bird_doutstring = ''
                tdoc_toks=[]
                tdoc_toks.extend(moses_tokenizer.tokenize(row['text'].replace('\n',' ')))    
                tdoc_toks.extend(['<eos>'])
                test_bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                vocab, len(tdoc_toks), 1, last_batch='keep')  #changed from 'discard'
                test_doc = test_bptt_batchify(tdoc_toks)
                for mix_model in mix_models:
                    dres = mevaluate(mix_model, test_doc, 1, context[0])
                    bird_doutstring= bird_doutstring +str(math.exp(dres))+","
                print(row['file'],',',bird_doutstring[:-1],file=g)


            for i,row in test_frame.iterrows():
                doutstring = ''
                tdoc_toks=[]
                tdoc_toks.extend(moses_tokenizer.tokenize(row['text'].replace('\n',' ')))    
                tdoc_toks.extend(['<eos>'])

                held_test_bptt = len(tdoc_toks)

                test_bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                vocab, held_test_bptt, test_batch_size, last_batch='keep')  #changed from 'discard'

                test_dbptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
                dvocab, held_test_bptt, test_batch_size, last_batch='keep')

                test_doc = test_bptt_batchify(tdoc_toks)
                
                for mix_model in mix_models:
                    dres = mevaluate(mix_model, test_doc, 1, context[0])
                    doutstring = doutstring + str(math.exp(dres)) + ','
                
                print(row['label'],',',doutstring[:-1],',',row['file'],file=f)

            f.flush()
            g.flush()
        f.close()
        g.close()

